In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import keras_tuner as kt
import tensorboard
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.python.client import device_lib 
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet101V2, Xception, InceptionResNetV2
from tensorflow.keras.applications import resnet_v2, xception, inception_resnet_v2
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical, Sequence

tf.random.set_seed(42)

global PROJECT_DIRECTORY
PROJECT_DIRECTORY = os.getcwd()

train_directory = "./data/organized/train/"
val_directory = "./data/organized/val/"
test_directory = "./data/organized/test/"

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
def create_tensorflow_datasets(image_size, train_directory, val_directory, test_directory, batch_size=32):
    
    train_dataset = image_dataset_from_directory(directory = train_directory,
                                                 labels='inferred',
                                                 label_mode = 'int',
                                                 image_size=image_size,
                                                 batch_size=batch_size,
                                                 smart_resize=True)

    val_dataset = image_dataset_from_directory(directory = val_directory,
                                               labels='inferred',
                                               label_mode = 'int',
                                               image_size=image_size,
                                               batch_size=batch_size,
                                               smart_resize=True)

    test_dataset = image_dataset_from_directory(directory = test_directory,
                                                labels = "inferred",
                                                label_mode = "int",
                                                image_size=image_size,
                                                batch_size=batch_size,
                                                smart_resize=True)
    
    return train_dataset, val_dataset, test_dataset

In [4]:
train_dataset, val_dataset, test_dataset = create_tensorflow_datasets(image_size=(520, 520),
                                                                      train_directory=train_directory,
                                                                      val_directory=val_directory,
                                                                      test_directory=test_directory,
                                                                      batch_size=32)

Found 10520 files belonging to 196 classes.
Found 3234 files belonging to 196 classes.
Found 2431 files belonging to 196 classes.


In [5]:
def calculate_base_model_outputs(tf_dataset, base_model_type, input_shape, output_pooling):
    
    valid_base_model_types = ['resnet101', 'xception', 'inception_resnet']
    
    if base_model_type not in valid_base_model_types:
        print("/n===========================================================")
        print("Invalid input for parameter base_model_type")
        print(f"Valid inputs are: {valid_base_model_types}")
        print("===========================================================\n")
        return -1
    
    target_labels = []
    model_output_features = []
    

    data_augmentation = keras.Sequential([layers.experimental.preprocessing.RandomFlip("horizontal"),
                                          layers.experimental.preprocessing.RandomRotation(0.1),
                                          layers.experimental.preprocessing.RandomZoom(0.2)])
    
    
    if base_model_type == 'resnet101':
        
        base_model =  ResNet101V2(weights='imagenet',
                                  input_shape = input_shape,
                                  include_top=False,
                                  pooling = output_pooling)
            
        for images, labels in tf_dataset:
                
            augmented_images = data_augmentation(images)

            preprocessed_images = resnet_v2.preprocess_input(augmented_images)

            features = base_model.predict(preprocessed_images)

            model_output_features.append(features)

            target_labels.append(labels)

        return np.concatenate(model_output_features), np.concatenate(target_labels)
        
    elif base_model_type == 'xception':
        
        base_model = Xception(weights='imagenet',
                              input_shape=input_shape,
                              include_top=False,
                              pooling = output_pooling)
        
        for images, labels in tf_dataset:
            
            augmented_images = data_augmentation(images)
            
            preprocessed_images = xception.preprocess_input(augmented_images)
            
            features = base_model.predict(preprocessed_images)
            
            model_output_features.append(features)
            
            target_labels.append(labels)
            
        return np.concatenate(model_output_features), np.concatenate(target_labels)
    
    elif base_model_type == 'inception_resnet':
        
        base_model = InceptionResNetV2(weights='imagenet',
                                       input_shape=input_shape,
                                       include_top=False,
                                       pooling = output_pooling)
        
        for images, labels in tf_dataset:
            
            augmented_images = data_augmentation(images)
            
            preprocessed_images = inception_resnet_v2.preprocess_input(augmented_images)
            
            features = base_model.predict(preprocessed_images)
            
            model_output_features.append(features)
            
            target_labels.append(labels)
            
        return (np.concatenate(model_output_features), np.concatenate(target_labels))

In [6]:
def save_base_model_outputs_to_disk(features, labels, base_model_type, input_shape, output_pooling, num_unique_epochs, current_epoch,
                                    dataset_type):
    
    global PROJECT_DIRECTORY
    
    # Directory that will contain all outputs from base_model_type (as many data augmented variations as we make) that are
    # preprocessed in the same way... (same starting image size and output pooling).
    base_dir_name = f"pretrained_model_output_features_numpy/{base_model_type}_pool{str(output_pooling)}_inShape_{str(input_shape)}_numpy"
    base_save_directory = os.path.join(PROJECT_DIRECTORY, base_dir_name)
    
    # Adding a unique folder inside the above directory that specifies if this is a train, val or test set
    full_dir_path = os.path.join(base_save_directory, f"{dataset_type}_unique_epoch_{current_epoch}_of_{num_unique_epochs}_numpy_")
    
    os.makedirs(full_dir_path, exist_ok = True)
    
    full_file_path = os.path.join(full_dir_path, f"{dataset_type}.npz")
    
    np.savez_compressed(file = full_file_path,
                        **{f"{dataset_type}_features" : features, f"{dataset_type}_labels" : labels})
    
    
    return

In [7]:
# tf_dataset, base_model_type, input_shape, output_pooling

def create_base_model_output_dirs(train_dataset, val_dataset, test_dataset, base_model_type, input_shape, output_pooling, num_unique_epochs,
                                  verbose = True):
    
    global_clock = time.time()
    
    dataset_types = ['train', 'val', 'test']
    datasets = [train_dataset, val_dataset, test_dataset]
    
    for epoch_num in range(1, num_unique_epochs + 1):
    
        for dataset, dset_type in zip(datasets, dataset_types):
            
            if verbose:
                start_time = time.time()
                print("\n=======================================================================================")
                print(f"Calculating pretrained {base_model_type} outputs for the {dset_type} dataset.")
                print(f"Processing epoch set {epoch_num } of {num_unique_epochs}")
                print(f"Output pooling setting: {output_pooling}")
                print(f"Size of images before {base_model_type} preprocessing: {input_shape}")
                print("=======================================================================================\n")
            
            output_features, labels = calculate_base_model_outputs(tf_dataset = dataset,
                                                                   base_model_type = base_model_type,
                                                                   input_shape = input_shape,
                                                                   output_pooling = output_pooling)
            
            if verbose:
                print("\n=======================================================================================")
                print(f"Finished calcuating {base_model_type} outputs!")
                print(f"Time spent getting features: {time.time() - start_time} seconds.")
                print("Starting to save outputs to a numpy array on disk...")
                save_start_time = time.time()
                print("=======================================================================================\n")
        
            save_base_model_outputs_to_disk(features = output_features,
                                            labels = labels,
                                            base_model_type = base_model_type,
                                            input_shape = input_shape,
                                            output_pooling = output_pooling,
                                            num_unique_epochs=num_unique_epochs,
                                            current_epoch = epoch_num,
                                            dataset_type = dset_type)
            if verbose:
                print("\n=======================================================================================")
                print("Finished saving to disk!")
                print(f"Time spent saving: {time.time() - save_start_time} seconds.")
                print("=======================================================================================\n")
                
    if verbose:
        print("\n=======================================================================================")
        print("Finished getting preprocessed features for all datasets and epochs!")
        print(f"Time spent: {round(((time.time() - global_clock) / 60), 2)} minutes.")
        print("=======================================================================================\n")
    

In [8]:

create_base_model_output_dirs(train_dataset = train_dataset,
                              val_dataset = val_dataset,
                              test_dataset = test_dataset,
                              base_model_type = 'resnet101',
                              input_shape = (520, 520, 3),
                              output_pooling = 'None',
                              num_unique_epochs = 1,
                              verbose = True)


Calculating pretrained resnet101 outputs for the train dataset.
Processing epoch set 1 of 1
Output pooling setting: None
Size of images before resnet101 preprocessing: (520, 520, 3)


Finished calcuating resnet101 outputs!
Time spent getting features: 385.53385853767395 seconds.
Starting to save outputs to a numpy array on disk...


Finished saving to disk!
Time spent saving: 270.0609202384949 seconds.


Calculating pretrained resnet101 outputs for the val dataset.
Processing epoch set 1 of 1
Output pooling setting: None
Size of images before resnet101 preprocessing: (520, 520, 3)


Finished calcuating resnet101 outputs!
Time spent getting features: 106.22245979309082 seconds.
Starting to save outputs to a numpy array on disk...


Finished saving to disk!
Time spent saving: 79.77779841423035 seconds.


Calculating pretrained resnet101 outputs for the test dataset.
Processing epoch set 1 of 1
Output pooling setting: None
Size of images before resnet101 preprocessing: (520, 520, 3)


Fi

In [9]:
# Reloading the numpy arrays.
'''
train_arrays = np.load(dir_path)
t_features = train_arrays['train_features']
t_labels = train_arrays['train_labels']



print(f"output_features shape {output_features.shape}")
print(f"labels shape: {labels.shape}")
''';